<a href="https://colab.research.google.com/github/bbishal12/Quiz0/blob/main/Exam_Review_Quiz_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Visualization
##Bishal Budhathoki
##Quiz 12: Exam Review


At this point you should be able to gather data from:

A CSV or Excel file
SQL databases
HTML tables
Twitter
From all that data that you have gathered you should be able to do some basic manipulations on it using pandas or sql langauge.

To prepare yourself for the exam, attempt the following problems:

Use the SQL table bigquery-public-data.london_fire_brigade.fire_brigade_service_calls
How many entries are there in the dataset
How many False Alarms happened between 10PM and 5 AM?
What is the average number of num_pumps_attending when there is a Fire?
What is the largest number of num_pumps_attending to a Road Vehicle property_category
Navigate to the Oklahoma wikipedia page. Grab the table on tempuratures in largest cities.
What is the average monthly temperature for OKC?
Create a plot of the average monthly tempurature in Tulsa.
Identify the city with the highest average tempurature in each month.

##My Solutions

SQL

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


How many total rows are there?

In [2]:
%%bigquery --project data-viz-bishal
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.london_fire_brigade.fire_brigade_service_calls`

,total_rows
0,32247


Here I am peaking at the data.

In [3]:
%%bigquery --project data-viz-bishal
SELECT *
FROM `bigquery-public-data.london_fire_brigade.fire_brigade_service_calls`
LIMIT 10

,incident_number,date_of_call,cal_year,time_of_call,hour_of_call,timestamp_of_call,incident_group,stop_code_description,special_service_type,property_category,property_type,address_qualifier,postcode_full,postcode_district,borough_code,borough_name,proper_case,ward_code,ward_name,ward_name_new,easting_m,northing_m,easting_rounded,northing_rounded,frs,incident_station_ground,first_pump_arriving_attendance_time,first_pump_arriving_deployed_from_station,second_pump_arriving_attendance_time,second_pump_arriving_deployed_from_station,num_stations_with_pumps_attending,num_pumps_attending
0,000320-01012017,2017-01-01,2017,15:30:22,15,2017-01-01 15:30:22+00:00,False Alarm,AFA,None,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,Within same building,None,IG9,E00000000,NOT GEO-CODED,Not geo-coded,E00000000,Not geo-coded,Not geo-coded,NaN,NaN,541450,194050,OverTheBorder,OverTheBorder,None,None,None,None,2,2
1,003727-09012017,2017-01-09,2017,10:41:54,10,2017-01-09 10:41:54+00:00,False Alarm,AFA,None,Dwelling,House - single occupancy,Correct incident location,None,IG7,E00000000,NOT GEO-CODED,Not geo-coded,E00000000,Not geo-coded,Not geo-coded,NaN,NaN,543650,192950,OverTheBorder,OverTheBorder,None,None,None,None,2,2
2,003762-09012017,2017-01-09,2017,12:28:39,12,2017-01-09 12:28:39+00:00,False Alarm,AFA,None,Dwelling,House - single occupancy,Correct incident location,None,IG7,E00000000,NOT GEO-CODED,Not geo-coded,E00000000,Not geo-coded,Not geo-coded,NaN,NaN,543650,192950,OverTheBorder,OverTheBorder,None,None,None,None,2,2
3,006403-16012017,2017-01-16,2017,04:02:14,4,2017-01-16 04:02:14+00:00,False Alarm,AFA,None,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,Correct incident location,None,IG7,E00000000,NOT GEO-CODED,Not geo-coded,E00000000,Not geo-coded,Not geo-coded,NaN,NaN,544350,192250,OverTheBorder,OverTheBorder,None,None,None,None,2,2
4,010947-26012017,2017-01-26,2017,11:14:03,11,2017-01-26 11:14:03+00:00,False Alarm,AFA,None,Dwelling,House - single occupancy,Correct incident location,None,IG10,E00000000,NOT GEO-CODED,Not geo-coded,E00000000,Not geo-coded,Not geo-coded,NaN,NaN,542050,196450,OverTheBorder,OverTheBorder,None,None,None,None,1,1
5,014169-03022017,2017-02-03,2017,04:58:02,4,2017-02-03 04:58:02+00:00,False Alarm,AFA,None,Dwelling,Converted Flat/Maisonettes - 3 or more storeys,Correct incident location,None,TN14,E00000000,NOT GEO-CODED,Not geo-coded,E00000000,Not geo-coded,Not geo-coded,NaN,NaN,548750,157750,OverTheBorder,OverTheBorder,None,None,None,None,3,5
6,016263-08022017,2017-02-08,2017,09:47:05,9,2017-02-08 09:47:05+00:00,False Alarm,AFA,None,Dwelling,House - single occupancy,Correct incident location,None,IG7,E00000000,NOT GEO-CODED,Not geo-coded,E00000000,Not geo-coded,Not geo-coded,NaN,NaN,544150,192550,OverTheBorder,OverTheBorder,None,None,None,None,2,2
7,016865-09022017,2017-02-09,2017,18:40:43,18,2017-02-09 18:40:43+00:00,False Alarm,AFA,None,Non Residential,Warehouse,Correct incident location,UB9 5HJ,UB9,E00000000,NOT GEO-CODED,Not geo-coded,E00000000,Not geo-coded,Not geo-coded,504201.0,188280.0,504250,188250,OverTheBorder,OverTheBorder,None,None,None,None,1,1
8,017092-10022017,2017-02-10,2017,10:07:53,10,2017-02-10 10:07:53+00:00,False Alarm,AFA,None,Non Residential,Factory,Correct incident location,UB9 5HJ,UB9,E00000000,NOT GEO-CODED,Not geo-coded,E00000000,Not geo-coded,Not geo-coded,504287.0,188271.0,504250,188250,OverTheBorder,OverTheBorder,None,None,None,None,1,1
9,024609-27022017,2017-02-27,2017,09:34:33,9,2017-02-27 09:34:33+00:00,False Alarm,AFA,None,Dwelling,Converted Flat/Maisonette - Up to 2 storeys,Correct incident location,None,CR3,E00000000,NOT GEO-CODED,Not geo-coded,E00000000,Not geo-coded,Not geo-coded,NaN,NaN,537650,155050,OverTheBorder,OverTheBorder,None,None,None,None,1,1


To find 'False Alarms' between certain hours, I will use the `WHERE` command and join several clauses carefully with `AND` and `OR`

In [4]:
%%bigquery --project data-viz-bishal
SELECT COUNT(*) as number_of_late_night_false_alarms
FROM `bigquery-public-data.london_fire_brigade.fire_brigade_service_calls`
WHERE incident_group = "False Alarm" AND (hour_of_call >= 22 OR hour_of_call <= 4)

,number_of_late_night_false_alarms
0,2825


If we want to see about when there is a fire, we will need to either restrict our data to then or possible GROUP BY I think I'll use the group by since I already did one with the WHERE.

In [6]:
%%bigquery --project data-viz-bishal
SELECT incident_group, AVG(num_pumps_attending) as avg_pumps_attending
FROM `bigquery-public-data.london_fire_brigade.fire_brigade_service_calls`
GROUP BY incident_group

,incident_group,avg_pumps_attending
0,False Alarm,1.725776
1,Fire,1.633012
2,Special Service,1.177192


#HTML

To navigate to the wikipedia page I'll need some other packages!

In [9]:
import requests
import pandas as pa
from bs4 import BeautifulSoup


r = requests.get('https://en.wikipedia.org/wiki/Oklahoma')
html_contents = r.text
html_soup = BeautifulSoup(html_contents,"lxml")
tables = html_soup.find_all('table',class_="wikitable")

df = pa.read_html(str(tables))[0]
df

,City,Unnamed: 1,Jan,Feb,March,April,May,June,July,Aug,Sept,Oct,Nov,Dec
0,Oklahoma City,Avg. high,50 °F(10 °C),55 °F(13 °C),63 °F(17 °C),73 °F(23 °C),80 °F(27 °C),88 °F(31 °C),94 °F(34 °C),93 °F(34 °C),85 °F(29 °C),73 °F(23 °C),62 °F(17 °C),51 °F(11 °C)
1,Oklahoma City,Avg. low,29 °F(−2 °C),33 °F(1 °C),41 °F(5 °C),50 °F(10 °C),60 °F(16 °C),68 °F(20 °C),72 °F(22 °C),71 °F(22 °C),63 °F(17 °C),52 °F(11 °C),40 °F(4 °C),31 °F(−1 °C)
2,Tulsa,Avg. high,48 °F(9 °C),53 °F(12 °C),62 °F(17 °C),72 °F(22 °C),79 °F(26 °C),88 °F(31 °C),93 °F(34 °C),93 °F(34 °C),84 °F(29 °C),73 °F(23 °C),61 °F(16 °C),49 °F(9 °C)
3,Tulsa,Avg. low,27 °F(−3 °C),31 °F(−1 °C),40 °F(4 °C),49 °F(9 °C),59 °F(15 °C),68 °F(20 °C),73 °F(23 °C),71 °F(22 °C),62 °F(17 °C),51 °F(11 °C),40 °F(4 °C),30 °F(−1 °C)
4,Lawton,Avg. high,50 °F(10 °C),56 °F(13 °C),65 °F(18 °C),73 °F(23 °C),82 °F(28 °C),90 °F(32 °C),96 °F(36 °C),95 °F(35 °C),86 °F(30 °C),76 °F(24 °C),62 °F(17 °C),52 °F(11 °C)
5,Lawton,Avg. low,26 °F(−3 °C),31 °F(−1 °C),40 °F(4 °C),49 °F(9 °C),59 °F(15 °C),68 °F(20 °C),73 °F(23 °C),71 °F(22 °C),63 °F(17 °C),51 °F(11 °C),39 °F(4 °C),30 °F(−1 °C)


So, these are the table on tempuratures in largest cities of Oklahoma.

In [10]:
df['Jan']

0    50 °F(10 °C)
1    29 °F(−2 °C)
2     48 °F(9 °C)
3    27 °F(−3 °C)
4    50 °F(10 °C)
5    26 °F(−3 °C)
Name: Jan, dtype: object

In [11]:
df.City == "Oklahoma City"

0     True
1     True
2    False
3    False
4    False
5    False
Name: City, dtype: bool

In [12]:
df[df.City == "Oklahoma City"]

,City,Unnamed: 1,Jan,Feb,March,April,May,June,July,Aug,Sept,Oct,Nov,Dec
0,Oklahoma City,Avg. high,50 °F(10 °C),55 °F(13 °C),63 °F(17 °C),73 °F(23 °C),80 °F(27 °C),88 °F(31 °C),94 °F(34 °C),93 °F(34 °C),85 °F(29 °C),73 °F(23 °C),62 °F(17 °C),51 °F(11 °C)
1,Oklahoma City,Avg. low,29 °F(−2 °C),33 °F(1 °C),41 °F(5 °C),50 °F(10 °C),60 °F(16 °C),68 °F(20 °C),72 °F(22 °C),71 °F(22 °C),63 °F(17 °C),52 °F(11 °C),40 °F(4 °C),31 °F(−1 °C)


So, these are the average monthly temperature for OKC.

In [13]:
df.query('City == "Tulsa"')

,City,Unnamed: 1,Jan,Feb,March,April,May,June,July,Aug,Sept,Oct,Nov,Dec
2,Tulsa,Avg. high,48 °F(9 °C),53 °F(12 °C),62 °F(17 °C),72 °F(22 °C),79 °F(26 °C),88 °F(31 °C),93 °F(34 °C),93 °F(34 °C),84 °F(29 °C),73 °F(23 °C),61 °F(16 °C),49 °F(9 °C)
3,Tulsa,Avg. low,27 °F(−3 °C),31 °F(−1 °C),40 °F(4 °C),49 °F(9 °C),59 °F(15 °C),68 °F(20 °C),73 °F(23 °C),71 °F(22 °C),62 °F(17 °C),51 °F(11 °C),40 °F(4 °C),30 °F(−1 °C)


In [7]:
import requests
import pandas as pa
from bs4 import BeautifulSoup


r = requests.get('https://en.wikipedia.org/wiki/Oklahoma')
html_contents = r.text
html_soup = BeautifulSoup(html_contents,"lxml")
tables = html_soup.find_all('table',class_="toccolours")

df = pa.read_html(str(tables))[0]


df.columns = df.columns.droplevel(0)
df

,Census,Pop.,Unnamed: 2_level_1,%±
0,1890,258657,NaN,—
1,1900,790391,NaN,205.6%
2,1910,1657155,NaN,109.7%
3,1920,2028283,NaN,22.4%
4,1930,2396040,NaN,18.1%
5,1940,2336434,NaN,−2.5%
6,1950,2233351,NaN,−4.4%
7,1960,2328284,NaN,4.3%
8,1970,2559229,NaN,9.9%
9,1980,3025290,NaN,18.2%
